In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde

#### ETAPA 1 : EDA

In [ ]:
# Descargamos el dataframe con información de los municipios
df_muni = pd.read_excel("base_municipios.xlsx")
df_muni.head(5)

In [ ]:
# Descargamos el dataframe con información de los inscriptos al programa
df_insc = pd.read_excel("ficha_inscriptos.xlsx")
df_insc.head(5)

In [ ]:
# Descargamos el dataframe con información de los formularios completados por los inscriptos
df_form = pd.read_excel("formularios_curso.xlsx")
df_form.head(5)

In [ ]:
# Concatenamos dataframes por columna
df_evaluate = pd.concat([df_muni, df_insc, df_form], axis=1)

In [ ]:
# Chequeamos los nombres de todas las columnas del dataframe concatenado
df_evaluate.columns.tolist()

In [ ]:
# Eliminamos columnas prescindibles, como 'municipio'
df_evaluate.drop(['municipio'], axis=1, inplace=True)

#### ETAPA 2 : Criterio de selección inicial

In [ ]:
# Chequeamos los valores únicos que clasifica etapas de inscripción
df_evaluate['etapa_inscripcion'].unique()

In [ ]:
# Chequeamos los valores únicos que clasifica estado de la solicitud 
df_evaluate['state'].unique()

In [ ]:
# Filtramos solo los inscriptos en etapa 1
formularios_estudio = df_evaluate[df_evaluate['etapa_inscripcion'] == 1 ]

In [ ]:
#Filtramos solo los inscriptos en estado solicitud_adjudicada ó solicitud_elegible_rechazadas_por_excedente 
formularios_estudio[formularios_estudio['state'] == 'solicitud_adjudicada']
formularios_estudio[formularios_estudio['state'] =='solicitud_elegible_rechazadas_por_excedente']

In [ ]:
# Asignamos variable al filtro de los inscriptos en estado solicitud_adjudicada ó solicitud_elegible_rechazadas_por_excedente
filtro = (formularios_estudio['state'] == 'solicitud_adjudicada') | \
         (formularios_estudio['state'] == 'solicitud_elegible_rechazadas_por_excedente')

# Creamos el nuevo dataframe aplicando el filtro
formularios_estudio = formularios_estudio[filtro]

In [ ]:
formularios_estudio

### ETAPA 3: Cálculo de atributos faltantes (edad)

In [ ]:
# Cálculo de edad

# Chequeamos la columna 'fecha_carga'
formularios_estudio['fecha_carga']

In [ ]:
#Chequeamos la columna 'fecha_de_nacimiento'
formularios_estudio['fecha_de_nacimiento']

In [ ]:
# Como los valores de cada columna son de distinto tipo de dato, transformamos 'fecha_de_nacimiento' para poder operar 
formularios_estudio['fecha_de_nacimiento'] = pd.to_datetime(
    formularios_estudio['fecha_de_nacimiento'], 
    errors='coerce'
)

In [ ]:
# Normalizamos quitando hora/minutos/segundo
formularios_estudio['fecha_de_nacimiento'] = formularios_estudio['fecha_de_nacimiento'].dt.normalize()

In [ ]:
# Obtenemos la diferencia entre 'fecha_de_nacimiento' y 'fecha_carga'
edad =  (formularios_estudio['fecha_carga'] - formularios_estudio['fecha_de_nacimiento']).dt.days
edad


In [ ]:
#df_inscriptos_etapa_1['edad'] = edad // 365.25
formularios_estudio['edad'] = (edad // 365.25)

# Convertir la columna 'edad' a tipo entero y mantiene valores NA
formularios_estudio['edad'] = pd.to_numeric(formularios_estudio['edad'], errors='coerce').astype('Int64')

In [ ]:
formularios_estudio['edad']

#### ETAPA 4 : Renombramiento

##### Crear una fila nueva cuyos elementos organice la columna 'state' en dos condiciones: si el valor es 'solicitud_adjudicada', se llamará "Tratamiento", de lo contrario, se llamará "Control" (es decir, los valores 'desestimado', 'no_admisible', 'admisible_fuera_de_cupo''para_enviar_a_use', 'presentado','solicitud_rechazada_por_falta_de_elegibilidad', 'enviada_a_use','admisible' 'solicitud_elegible_rechazada_por_excedente','solicitud_con_adjudicacion_iniciada' 'solicitud_elegible''solicitud_elegible_rechazadas_por_excedente', estarán en este último grupo).

In [ ]:
# Hacemos cálculos estadísticos básicos sobre la columna 'ingreso_anual_hogar'
ejemplo_agrupaciones_2 = formularios_estudio['ingreso_anual_hogar'].agg(
    cantidad='count',
    media_ing='mean',
    max_ing='max',
    min_ing='min',
    desvio_ing='std'
).to_frame().T

In [ ]:
ejemplo_agrupaciones_2

In [ ]:
# Calculamos el coeficiente de variación
ejemplo_agrupaciones_3 = formularios_estudio['ingreso_anual_hogar'].agg(
    cantidad='count',
    media_ing='mean',
    desvio_ing='std',
).to_frame().T

In [ ]:
ejemplo_agrupaciones_3['%desv_ing'] = (ejemplo_agrupaciones_3['desvio_ing'] / ejemplo_agrupaciones_3['media_ing'] * 100).round(1)

In [ ]:
ejemplo_agrupaciones_3

In [ ]:
# el desvío estándar es mucho mayor que la media.

In [ ]:
# Calcula la proporción de género femenino
ejemplo_agrupaciones_4 = pd.DataFrame({
    '%sexo_f': [(formularios_estudio['sexo_dni'] == 'F').mean()]
})

In [ ]:
ejemplo_agrupaciones_4

In [ ]:
#Análisis total

# Implementamos una función para el cálculo de media y desviación estándar
def calcular_media_std(df:pd.DataFrame, columna:str):
    return (
        f'La media de {columna} es {float(round(df[columna].mean(skipna=True), 1))} ',
        f'La desviación estándar de {columna} es {float(round((df[columna].std(skipna=True)), 1))}'
    )

In [ ]:
calcular_media_std(formularios_estudio, 'ingreso_anual_hogar')

In [ ]:
calcular_media_std(formularios_estudio, 'edad')

In [ ]:
calcular_media_std(formularios_estudio, 'personas_por_ambiente')

In [ ]:
# Implementamos una función para el cálculo de media bajo una condición

def calcular_media_condicion(df: pd.DataFrame, columna: str, condicion: str):
    proporcion = (df[columna] == condicion).mean(skipna=True)
    print(f" La proporción de la media de {condicion} en {columna} es {round(proporcion, 1)}")
    return round(float(proporcion), 1)

In [ ]:
calcular_media_condicion(formularios_estudio,'sexo_dni', 'F')
calcular_media_condicion(formularios_estudio, 'sexo_dni', 'M')
calcular_media_condicion(formularios_estudio, 'relacion_de_parentezco_con_jefe_del_hogar', 'Soy jefa(e)')
calcular_media_condicion(formularios_estudio, 'conurbano_interior', 'Conurbano')

In [ ]:
#Chequeamos el motivo que de la proporción de la media en 'conurbano_interior'es 0.0
(formularios_estudio['conurbano_interior'] == 'Conurbano').value_counts()


In [ ]:
vuln = float(round(formularios_estudio['escenario_vulnerabilidad_social'].mean(skipna=True), 1))
vuln

In [ ]:
# Se agrupa en 'Tratamiento' si se cumple 'solicitud_adjudicada', y 'control', en caso contrario
formularios_estudio['grupo'] = np.where( formularios_estudio['state'] == 'solicitud_adjudicada','Tratamiento', 'Control')

formularios_estudio['grupo']

In [ ]:
# Guardar extracción de datos en "Control" o "Tratamiento"
grupo_control = (formularios_estudio['grupo']) == 'Control' # es una serie que devuelve booleanos
grupo_tratamiento = (formularios_estudio['grupo']) == 'Tratamiento' # idem

In [ ]:
# A cada grupo le damos estructura de dataframe nuevo
df_control = formularios_estudio[grupo_control]
df_tratamiento = formularios_estudio[grupo_tratamiento]

In [ ]:
# Se eliminan columnas innecesarias en ambos dataframes
df_control_clean = df_control.drop(columns=[
    'codigo_municipio', 'codigo_region', 'nombre_region', 'codigo_area',
    'nombre_area', 'conurbano_interior', 'seccion_electoral',
    'superficie(km2)', 'intendente', 'partido_politico_actual', 
    'poblacion_censo_2010', 'poblacion_censo_2022'
])


In [ ]:
df_tratamiento_clean = df_tratamiento.drop(columns=[
    'codigo_municipio', 'codigo_region', 'nombre_region', 'codigo_area',
    'nombre_area', 'conurbano_interior', 'seccion_electoral',
    'superficie(km2)', 'intendente', 'partido_politico_actual', 
    'poblacion_censo_2010', 'poblacion_censo_2022'
])

In [ ]:
# Creamos nueva función que calcula media y desviación estándar

def calcular_media_std_1(df: pd.DataFrame, columna:str):
    media = round(df[columna].mean(skipna=True),1)
    std = round(df[columna].std(skipna=True),1)
    return media, std

In [ ]:
# Creamos una función que utiliza el cálculo anterior, y transformaciones a dataframe
def calcular_media_std_lista(df: pd.DataFrame, lista: list):
    resultados = {}
    for columna in lista:
        media, std = calcular_media_std_1(df, columna)
        resultados[f'media_{columna}'] = media
        resultados[f'desv_{columna}'] = std
    return pd.DataFrame([resultados]) # pandas necesita lista de diccionarios


In [ ]:
lista = ['ingreso_anual_hogar', 'edad', 'personas_por_ambiente']
calcular_media_std_lista(df_tratamiento_clean, lista)

In [ ]:
lista = ['ingreso_anual_hogar', 'edad', 'personas_por_ambiente']
calcular_media_std_lista(df_control_clean, lista)

In [ ]:
def calcular_media_condicion(df: pd.DataFrame, dict_condiciones: dict):
    resultado = {}
    for columna, lista_condiciones in dict_condiciones.items():
        for condicion in lista_condiciones:
            proporcion = (df[columna] == condicion).mean(skipna=True)
            resultado[f'media_{columna}_{condicion}'] = round(proporcion,1)
    return pd.DataFrame([resultado])

In [ ]:
dict_condiciones = {
    'sexo_dni': ['F','M'],
    'relacion_de_parentezco_con_jefe_del_hogar': ['Soy jefa(e)']
}

calcular_media_condicion(df_tratamiento_clean, dict_condiciones)

In [ ]:
calcular_media_condicion(df_control_clean, dict_condiciones)

In [ ]:
# Resetear los índices de cada dataframe por muestra

In [ ]:
# Grupo tratamiento

df_media_tratamiento = calcular_media_condicion(df_tratamiento_clean, dict_condiciones)
df_media_std_tratamiento = calcular_media_std_lista(df_tratamiento_clean,lista)

df1 = df_media_tratamiento.reset_index(drop=True)
df2 = df_media_std_tratamiento.reset_index(drop=True)

In [ ]:
df_tratamiento_total = pd.concat([df1, df2], axis=1)

In [ ]:
df_tratamiento_total

In [ ]:
# Grupo control

df_media_control = calcular_media_condicion(df_control_clean, dict_condiciones)
df_media_std_control = calcular_media_std_lista(df_control_clean,lista)

df3 = df_media_control.reset_index(drop=True)
df4 = df_media_std_control.reset_index(drop=True)


In [ ]:
df_control_total = pd.concat([df3, df4], axis=1)
df_control_total

In [ ]:
df_analisis = pd.concat([df_tratamiento_total, df_control_total], axis=0)
df_analisis


In [ ]:
vuln_control = float(round(df_control_clean['escenario_vulnerabilidad_social'].mean(skipna=True), 1))
vuln_control

In [ ]:
fila_vuln_c = pd.DataFrame({'escenario_vulnerabilidad_social': [vuln_control]}, index= [1])
fila_vuln_c

In [ ]:
vuln_tratamiento = float(round(df_tratamiento_clean['escenario_vulnerabilidad_social'].mean(skipna=True), 1))
vuln_tratamiento

In [ ]:
fila_vuln_t = pd.DataFrame({'escenario_vulnerabilidad_social': [vuln_tratamiento]}, index= [1])
fila_vuln_t

In [ ]:
df_vuln = pd.concat([fila_vuln_t, fila_vuln_c], axis=0)

In [ ]:
df_analisis = df_analisis.reset_index(drop=True)

In [ ]:
df_vuln = df_vuln.reset_index(drop=True)

In [ ]:
df_analisis_total = pd.concat([df_analisis, df_vuln], axis= 1)

In [ ]:
df_analisis_total

In [ ]:
# Agregamos una columna nueva para aclarar las muestras
df_analisis_total.insert(loc=0, column='Grupo', value=['Tratamiento', 'Control'])

In [ ]:
df_analisis_total

In [ ]:
# Crear vector de identificadores



#### ETAPA 5: PLOT de Distribución de edad por estado"

In [ ]:
# Configurar bins
bins = np.arange(0, 95, 5)
bin_labels = [f'{i}-{i+5}' for i in bins[:-1]]

# Paleta categórica apta para daltónicos
palette = sns.color_palette("colorblind", len(bins) - 1)

# Crear figura
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
grupos = ['Control', 'Tratamiento']

# Limpiar valores fuera del rango lógico (0 a 100)
formularios_estudio = formularios_estudio[(formularios_estudio['edad'] >= 0) & (formularios_estudio['edad'] <= 100)]

for i, grupo in enumerate(grupos):
    ax = axes[i]
    subset = formularios_estudio[formularios_estudio['grupo'] == grupo]
    edades = subset['edad']

    # Asignar edades a bins
    bin_indices = np.digitize(edades, bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(palette) - 1)  # Evitar out of range
    
    # Contar frecuencias por bin
    counts = np.zeros(len(bins) - 1)
    for idx in bin_indices:
        counts[idx] += 1
    
    # Dibujar barras para asignar color categórico
    for j, count in enumerate(counts):
        ax.bar(bins[j], count, width=5, align='edge',
               color=palette[j], edgecolor='white', linewidth=1.2)
    
    # mediana
    mediana = np.median(edades)
    ax.axvline(mediana, color='black', linestyle='--', linewidth=2, label=f'Mediana: {mediana:.1f}')
    
    # Estética
    ax.set_title(f'Distribución de Edad - {grupo}', fontsize=14)
    ax.set_xlabel('Edad', fontsize=12)
    ax.set_xlim(0, 90)
    ax.set_xticks(np.arange(0, 95, 10))
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    if i == 0:
        ax.set_ylabel('Frecuencia', fontsize=12)
    ax.legend(frameon=True)

# Crear leyenda de colores por bin
handles = [plt.Rectangle((0,0),1,1, color=palette[i]) for i in range(len(bin_labels))]
fig.legend(handles, bin_labels, title='Rango Etario', bbox_to_anchor=(1.05, 0.5), loc='center left')

# Título
plt.suptitle('Distribución de Edad por Grupo', fontsize=16, y=1.03)
plt.tight_layout()
plt.subplots_adjust(right=0.85)

plt.show()

In [ ]:
# Configurar bins
bins = np.arange(0, 95, 5)
bin_labels = [f'{i}-{i+5}' for i in bins[:-1]]

# Paleta categórica apta para daltónicos
palette = sns.color_palette("colorblind", len(bins) - 1)

# Crear figura
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
grupos = ['Control', 'Tratamiento']

# Limpiar valores fuera del rango lógico (0 a 100)
formularios_estudio = formularios_estudio[(formularios_estudio['edad'] >= 0) & (formularios_estudio['edad'] <= 100)]

for i, grupo in enumerate(grupos):
    ax = axes[i]
    subset = formularios_estudio[formularios_estudio['grupo'] == grupo]
    edades = subset['edad']

    
    # Asignar edades a bins
    bin_indices = np.digitize(edades, bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(palette) - 1)  # Evitar out of range
    
    # Contar frecuencias por bin
    counts = np.zeros(len(bins) - 1)
    for idx in bin_indices:
        counts[idx] += 1
    

    # Dibujar barras para asignar color categórico
    for j, count in enumerate(counts):
        ax.bar(bins[j], count, width=5, align='edge',
               color=palette[j], edgecolor='white', linewidth=1.2)



    # KDE suavizada 
    kde = gaussian_kde(edades, bw_method='scott')  
    x_vals = np.linspace(min(edades), max(edades), 200)
    y_vals = kde(x_vals)

    # Escalar la KDE a frecuencia absoluta
    bin_width = bins[1] - bins[0]
    y_vals *= len(edades) * bin_width

    ax.plot(x_vals, y_vals, color='deeppink', linewidth=2, label='KDE')


    # mediana
    mediana = np.median(edades)
    ax.axvline(mediana, color='black', linestyle='--', linewidth=2, label=f'Mediana: {mediana:.1f}')
    
    # Estética
    ax.set_title(f'Distribución de Edad - {grupo}', fontsize=14)
    ax.set_xlabel('Edad', fontsize=12)
    ax.set_xlim(0, 90)
    ax.set_xticks(np.arange(0, 95, 10))
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    if i == 0:
        ax.set_ylabel('Frecuencia', fontsize=12)
    ax.legend(frameon=True)

# Crear leyenda de colores por bin
handles = [plt.Rectangle((0,0),1,1, color=palette[i]) for i in range(len(bin_labels))]
fig.legend(handles, bin_labels, title='Rango Etario', bbox_to_anchor=(1.05, 0.5), loc='center left')

# Título
plt.suptitle('Distribución de Edad por Grupo', fontsize=16, y=1.03)
plt.tight_layout()
plt.subplots_adjust(right=0.85)

plt.show()

In [ ]:
# Configurar bins
bins = np.arange(0, 95, 5)
bin_labels = [f'{i}-{i+5}' for i in bins[:-1]]

# Paleta categórica apta para daltónicos
palette = sns.color_palette("colorblind", len(bins) - 1)

# Crear figura
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
grupos = ['Control', 'Tratamiento']

# Limpiar valores fuera del rango lógico (0 a 100)
formularios_estudio = formularios_estudio[(formularios_estudio['edad'] >= 0) & (formularios_estudio['edad'] <= 100)]

colores_kde = ['#FF7F50', '#6A5ACD']

for i, grupo in enumerate(grupos):
    ax = axes[i]
    subset = formularios_estudio[formularios_estudio['grupo'] == grupo]
    edades = subset['edad']
         
    # Asignar edades a bins
    bin_indices = np.digitize(edades, bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(palette) - 1)  # Evitar out of range
    
    # Contar frecuencias por bin
    counts = np.zeros(len(bins) - 1)
    for idx in bin_indices:
        counts[idx] += 1
    
    # Dibujar barras para asignar color categórico
    for j, count in enumerate(counts):
        ax.bar(bins[j], count, width=5, align='edge',
               color=palette[j], edgecolor='white', linewidth=1.2)


    # KDE suavizada 
    kde = gaussian_kde(edades, bw_method='scott')  
    x_vals = np.linspace(min(edades), max(edades), 200)
    y_vals = kde(x_vals)

    # Escalar la KDE a frecuencia absoluta
    bin_width = bins[1] - bins[0]
    y_vals *= len(edades) * bin_width

    ax.plot(x_vals, y_vals, color='deeppink', linewidth=2, label='KDE')

    #KDE área
    ax.fill_between(x_vals, y_vals, color=colores_kde[i], alpha=0.8, label='Área KDE')


    # mediana
    mediana = np.median(edades)
    ax.axvline(mediana, color='black', linestyle='--', linewidth=2, label=f'Mediana: {mediana:.1f}')
    
    # Estética
    ax.set_title(f'Distribución de Edad - {grupo}', fontsize=14)
    ax.set_xlabel('Edad', fontsize=12)
    ax.set_xlim(0, 90)
    ax.set_xticks(np.arange(0, 95, 10))
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    if i == 0:
        ax.set_ylabel('Frecuencia', fontsize=12)
    ax.legend(frameon=True)

# Crear leyenda de colores por bin
handles = [plt.Rectangle((0,0),1,1, color=palette[i]) for i in range(len(bin_labels))]
fig.legend(handles, bin_labels, title='Rango Etario', bbox_to_anchor=(1.05, 0.5), loc='center left')

# Título
plt.suptitle('Distribución de Edad por Grupo', fontsize=16, y=1.03)
plt.tight_layout()
plt.subplots_adjust(right=0.85)

plt.show()

In [ ]:
# mostrar plot de áreas KDE superpuestas

#### ETAPA 6 : MUESTRAS ALEATORIAS SIMPLES

In [ ]:
# Crear una función para calcular muestra
def muestra_aleatoria_simple(df= pd.DataFrame, n=int, seed=42 ):
    return df.sample(n=n,random_state=seed)

In [ ]:
# Generar muestras aleatorias del grupo control y tratamiento 
mas_control = muestra_aleatoria_simple(df_control_clean, 1000)
mas_tratamiento = muestra_aleatoria_simple (df_tratamiento_clean,1000)

In [ ]:
#Se combinan los resultados de las 1000 muestras del grupo Tratamiento con las del grupo Control
muestra_combinada = pd.concat([mas_control, mas_tratamiento], ignore_index=True)
muestra_combinada

In [ ]:
#Evaluar qué tan representativa es cada muestra usando bootstrapping
# Grupo Control
bootstrap_meds_c = []
for i in range(10000):
    sample = df_control_clean.sample(n=len(mas_control), replace=True)
    bootstrap_meds_c.append(sample['ingreso_anual_hogar'].mean())

In [ ]:
bootstrap_meds_c

In [ ]:
 # Grupo Tratamiento
bootstrap_meds_t = []
for i in range(10000):
    sample = df_tratamiento_clean.sample(n=len(mas_tratamiento), replace=True)
    bootstrap_meds_t.append(sample['ingreso_anual_hogar'].mean())

In [ ]:
bootstrap_meds_t

In [ ]:
#Calcular la media poblacional
media_poblacional = formularios_estudio['ingreso_anual_hogar'].mean()
media_poblacional

In [ ]:
# Coeficiente de representatividad de la muestra
    #Calcular la media del bootstrap_meds_c y el error relativo

media_bootstrap_c = np.mean(bootstrap_meds_c)

In [ ]:
# Error absoluto
error_absoluto_c = abs(media_bootstrap_c - media_poblacional)

In [ ]:
# Error relativo
error_relativo_c = error_absoluto_c / media_poblacional
error_relativo_c

In [ ]:
#Calcular la media del bootstrap_meds_t y el error relativo
media_bootstrap_t = np.mean(bootstrap_meds_t)

In [ ]:
# Error absoluto
error_absoluto_t = abs(media_bootstrap_t - media_poblacional)

In [ ]:
# Error relativo
error_relativo_t = error_absoluto_t / media_poblacional
error_relativo_t

In [ ]:
resultados_totales = pd.concat([bootstrap_meds_c, bootstrap_meds_t], ignore_index=True)

In [ ]:
# Elegir la muestra más representativa por grupo

#mejores_resultados = resultados_totales.loc[
#    resultados_totales.groupby("grupo")["ev_med_ingreso"].idxmin()
#].reset_index(drop=True)


In [ ]:
# Unir al análisis total para comparar

#analisis_total["grupo"] = "Total"

#total_optimizado = pd.concat([analisis_total, mejores_resultados], ignore_index=True)
#total_optimizado = total_optimizado[["grupo", "cantidad", "med_ing", "ev_med_ingreso"]]

In [ ]:
# Mostrar tabla resumida

#print(total_optimizado.round(6).to_markdown(index=False))

In [ ]:
# Convertir la lista de identificadores en texto plano

In [ ]:
# Descargamos el dataframe con información de los mejores resultados
df_mejores_resultados = pd.read_excel("mejores_resultados.xlsx")
df_mejores_resultados.head(5)

In [ ]:
# Descargamos el archivo de la nueva encuesta a hogares
df_nueva_encuesta = pd.read_excel("nueva_encuesta_situacion_hogar.xlsx")
df_nueva_encuesta.head(5)